In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tiktoken cohere
!pip install openai==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
# @title OpenAI initialization
from openai import OpenAI
client = OpenAI(api_key="<API_KEY>")

# @title Retrive Fine Tune Job
response = client.fine_tuning.jobs.retrieve("ftjob-<FINE_TUNE_JOB_ID>")
response2 = client.fine_tuning.jobs.retrieve("ftjob-<FINE_TUNE_JOB_ID>")
fine_tuned_model_id = response.fine_tuned_model
fine_tuned_model_id2 = response2.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)
print("\nFine-tuned model id2:", fine_tuned_model_id2)


Fine-tuned model id: ft:gpt-3.5-turbo-1106:personal::8i8a6GDi

Fine-tuned model id2: ft:gpt-3.5-turbo-0613:personal::8jN2f6OM


In [ ]:
# @title Sentiment analysis of a review

import json
import re
import pandas as pd

DEFAULT_SYSTEM_PROMPT = "Return dissatisfaction categories en each reviews as json. Dissatisfaction categories must be one of them: Content, UI/UX, Bugs/stability, Customer service, Subscription/payment, Downloading/connection. Do not return any result for a review if there is no dissatisfaction with these categories. A review can be related to multiple categories at once. You are given an reviews object which the key is id and the value is review. You must return an object which the key is id and the value is string array. Each review should be analyzed independently of the other."

def predict_with_llm(message, model):
    message_formatted = [{"role": "system", "content": DEFAULT_SYSTEM_PROMPT},{"role": "user", "content": message}]
    response = client.chat.completions.create(model=model, messages = message_formatted)
    return response.choices[0].message.content


# @markdown ### Enter review id and text
input_id = 11 # @param {type:"integer"}
input_review = "UI/UX is awesome but app have many bugs!"  # @param {type:"string"}
# @markdown ---
# @markdown ### Choose LLM model to use
which_model = 'GPT-3.5 1106' # @param ['GPT-3.5 0613', 'GPT-3.5 1106']

msg = {input_id: input_review}

msg_formatted = re.sub(r':\{(.*?)\}', r'\1:', json.dumps(msg))

selected_model_id = fine_tuned_model_id2 if(which_model == 'GPT-3.5 0613') else fine_tuned_model_id

results = predict_with_llm(msg_formatted, selected_model_id)

results_formatted = re.sub(r':\{(.*?)\}', r':[\1]', results)

results_arr = [{"id": input_id, "review": input_review, "result": json.loads(results_formatted)[str(input_id)]}]

df_results = pd.DataFrame(results_arr)

df_results = df_results.set_axis(df_results['id'], copy = True)
df_results.drop(columns=['id'], inplace=True)

df_results

review            result
id                                                            
11  UI/UX is awesome but app have many bugs!  [Bugs/stability]

In [ ]:
# @title Filter results by date and list filtered results.

import json
import pandas as pd
from datetime import datetime

to_what = 250_000

# output.json dosyasını oku
with open(f'/content/drive/MyDrive/0-{to_what}_merged.json', 'r', encoding='utf-8') as f:
    output_data = json.load(f)

df = pd.DataFrame(output_data)

df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.set_axis(df['id'], copy = True)
df.drop(columns=['id'], inplace=True)

# @markdown ### Set results date range
from_date = '2023-03-22' # @param {type:"date"}
until_date = '2023-03-23' # @param {type:"date"}
# @markdown ---
# @markdown ### Choose filter, or ignore selecting 'None'
filter_by = 'Downloading/connection' # @param ['None', 'Content','Customer service', 'Bugs/stability', 'UI/UX', 'Subscription/payment', 'Downloading/connection', 'Any']

d1 = datetime.strptime(from_date, '%Y-%M-%d')
d2 = datetime.strptime(until_date, '%Y-%M-%d')

if filter_by != 'None':
    if filter_by == 'Any':
        # 'Any' seçildiyse, result sütunundaki listelerin boş olmamasını kontrol et
        df = df[df['result'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
    else:
        # Diğer durumlarda, result sütunundaki listelerde filter_by kategorisinin olup olmadığını kontrol et
        df = df[df['result'].apply(lambda x: isinstance(x, list) and filter_by in x)]

df[(df["timestamp"] > d1) & (d2 > df["timestamp"])]

timestamp                                             review  \
id                                                                              
260470 2023-01-22 05:09:43                                     Loads too long   
260527 2023-01-22 11:26:59  The way Netflix is working on my new phone is ...   
260571 2023-01-22 14:59:04  Phone Lang gumagana Hindi sa TV nag plan pa na...   
260592 2023-01-22 15:50:32  I usually enjoy watching on Netflix when I do ...   
260599 2023-01-22 16:22:12  Fairly good app. But lately it keeps deleting ...   
260669 2023-01-22 22:53:34           IL ne peut pas installer Sur ma tablette   

                          result  
id                                
260470  [Downloading/connection]  
260527  [Downloading/connection]  
260571  [Downloading/connection]  
260592  [Downloading/connection]  
260599  [Downloading/connection]  
260669  [Downloading/connection]